# TL;DR

1. In this lab scenario you will have a chance to compare performance of the classic RNN and LSTM on a toy example. 
2. This toy example will show that maintaining memory over even 20 steps is non-trivial. 
3. Finally, you will see how curriculum learning may allow to train a model on larger sequences.

# Problem definition

Here we consider a toy example, where the goal is to discriminate between two types of binary sequences:
* [Type 0] a sequence with exactly one zero (remaining entries are equal to one).
* [Type 1] a sequence full of ones,

We are especially interested in the performance of the trained models on discriminating between a sequence full of ones versus a sequence with leading zero followed by ones. Note that in this case the goal of the model is to output the first element of the sequence, as the label (sequence type) is fully determined by the first element of the sequence.

#Implementation

## Importing torch

Install `torch` and `torchvision`

In [1]:
!pip3 install torch torchvision

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

torch.manual_seed(1)

## Understand dimensionality

Check the input and output specification [LSTM](https://pytorch.org/docs/stable/generated/torch.nn.LSTM.html) and [RNN](https://pytorch.org/docs/stable/generated/torch.nn.RNN.html). The following snippet shows how we can process
a sequence by LSTM and output a vector of size `hidden_dim` after reading
each token of the sequence. 

In [3]:
hidden_dim = 5
lstm = nn.LSTM(1, hidden_dim)  # Input sequence contains elements - vectors of size 1

# create a random sequence
sequence = [torch.randn(1) for _ in range(10)]

# initialize the hidden state (including cell state)
hidden = (torch.zeros(1, 1, 5),
          torch.zeros(1, 1, 5))

for i, elem in enumerate(sequence):
  # we are processing only a single element of the sequence, and there
  # is only one sample (sequence) in the batch, the third one
  # corresponds to the fact that our sequence contains elemenents,
  # which can be treated as vectors of size 1
  out, hidden = lstm(elem.view(1, 1, 1), hidden)
  print(f'i={i} out={out.detach()}')
print(f'Final hidden state={hidden[0].detach()} cell state={hidden[1].detach()}')

i=0 out=tensor([[[-0.0675,  0.1179,  0.1081,  0.0414, -0.0341]]])
i=1 out=tensor([[[-0.1067,  0.1726,  0.1400,  0.0902, -0.0596]]])
i=2 out=tensor([[[-0.1148,  0.1885,  0.1956,  0.0974, -0.0840]]])
i=3 out=tensor([[[-0.1270,  0.2031,  0.1495,  0.1249, -0.0860]]])
i=4 out=tensor([[[-0.1281,  0.2019,  0.1810,  0.1475, -0.1027]]])
i=5 out=tensor([[[-0.1274,  0.2060,  0.0798,  0.1330, -0.0860]]])
i=6 out=tensor([[[-0.1318,  0.2039,  0.0997,  0.1772, -0.1011]]])
i=7 out=tensor([[[-0.1145,  0.2008, -0.0431,  0.1051, -0.0717]]])
i=8 out=tensor([[[-0.1289,  0.1989,  0.0515,  0.1944, -0.1030]]])
i=9 out=tensor([[[-0.1329,  0.1920,  0.0686,  0.1772, -0.0988]]])
Final hidden state=tensor([[[-0.1329,  0.1920,  0.0686,  0.1772, -0.0988]]]) cell state=tensor([[[-0.2590,  0.4080,  0.1307,  0.4329, -0.2895]]])


## To implement

Process the whole sequence all at once by calling `lstm` only once and check that the output is exactly the same as above (remember to initialize the hidden state the same way).

In [4]:
inputs = torch.cat(sequence).view(len(sequence), 1, -1)
hidden = (torch.zeros(1, 1, 5),
          torch.zeros(1, 1, 5))
out, hidden = lstm(inputs, hidden)
print(f'Out={out.detach()}')
print(f'Final hidden state={hidden[0].detach()} cell state={hidden[1].detach()}')

Out=tensor([[[-0.0675,  0.1179,  0.1081,  0.0414, -0.0341]],

        [[-0.1067,  0.1726,  0.1400,  0.0902, -0.0596]],

        [[-0.1148,  0.1885,  0.1956,  0.0974, -0.0840]],

        [[-0.1270,  0.2031,  0.1495,  0.1249, -0.0860]],

        [[-0.1281,  0.2019,  0.1810,  0.1475, -0.1027]],

        [[-0.1274,  0.2060,  0.0798,  0.1330, -0.0860]],

        [[-0.1318,  0.2039,  0.0997,  0.1772, -0.1011]],

        [[-0.1145,  0.2008, -0.0431,  0.1051, -0.0717]],

        [[-0.1289,  0.1989,  0.0515,  0.1944, -0.1030]],

        [[-0.1329,  0.1920,  0.0686,  0.1772, -0.0988]]])
Final hidden state=tensor([[[-0.1329,  0.1920,  0.0686,  0.1772, -0.0988]]]) cell state=tensor([[[-0.2590,  0.4080,  0.1307,  0.4329, -0.2895]]])


## Training a model

Below we define a very simple model, which is a single layer of LSTM, where the output in each time step is processed by relu followed by a single fully connected layer, the output of which is a single number. We are going
to use the number generated after reading the last element of the sequence,
which will serve as the logit for our classification problem.

In [5]:
class Model(nn.Module):

    def __init__(self, hidden_dim):
        super(Model, self).__init__()
        self.hidden_dim = hidden_dim
        self.lstm = nn.LSTM(1, self.hidden_dim)
        self.hidden2label = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        out, _ = self.lstm(x)
        sequence_len = x.shape[0]
        logits = self.hidden2label(F.relu(out[-1].view(-1)))
        return logits

Below is a training loop, where we only train on the two hardest examples.

In [6]:
SEQUENCE_LEN = 10

# Pairs of (sequence, label)
HARD_EXAMPLES = [([0.]+(SEQUENCE_LEN-1)*[1.], 0),
                 (SEQUENCE_LEN*[1.], 1)]

def eval_on_hard_examples(model):
    with torch.no_grad():
        logits = []
        for sequence in HARD_EXAMPLES:
            input = torch.tensor(sequence[0]).view(-1, 1, 1)
            logit = model(input)
            logits.append(logit.detach())
        print(f'Logits for hard examples={logits}')


def train_model(hidden_dim, lr, num_steps=10000):
    model = Model(hidden_dim=hidden_dim)
    loss_function = nn.BCEWithLogitsLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.99)

    for step in range(num_steps):  
        if step % 100 == 0:
            eval_on_hard_examples(model)

        for sequence, label in HARD_EXAMPLES:
            model.zero_grad()
            logit = model(torch.tensor(sequence).view(-1, 1, 1))  
            
            loss = loss_function(logit.view(-1), torch.tensor([label], dtype=torch.float32))
            loss.backward()

            optimizer.step()   

In [7]:
train_model(hidden_dim=20, lr=0.01, num_steps=10000)

Logits for hard examples=[tensor([-0.1775]), tensor([-0.1774])]
Logits for hard examples=[tensor([0.0095]), tensor([0.0097])]
Logits for hard examples=[tensor([0.0259]), tensor([0.0263])]
Logits for hard examples=[tensor([-0.0017]), tensor([-0.0009])]
Logits for hard examples=[tensor([-0.0034]), tensor([7.1838e-05])]
Logits for hard examples=[tensor([-3.0632]), tensor([3.5531])]
Logits for hard examples=[tensor([-11.0094]), tensor([10.8645])]
Logits for hard examples=[tensor([-12.1567]), tensor([11.9191])]
Logits for hard examples=[tensor([-12.3136]), tensor([12.0644])]
Logits for hard examples=[tensor([-12.3368]), tensor([12.0874])]
Logits for hard examples=[tensor([-12.3420]), tensor([12.0939])]
Logits for hard examples=[tensor([-12.3448]), tensor([12.0982])]
Logits for hard examples=[tensor([-12.3473]), tensor([12.1022])]
Logits for hard examples=[tensor([-12.3498]), tensor([12.1062])]
Logits for hard examples=[tensor([-12.3523]), tensor([12.1101])]
Logits for hard examples=[tensor(

## To implement

1. Check for what values of `SEQUENCE_LEN` the model is able to discriminate betweeh the two hard examples (after training).
2. Instead of training on `HARD_EXAMPLES` only, modify the training loop to train on sequences where zero may be in any position of the sequence (so any valid sequence of `Type 0`, not just the hardest one). After modifying the training loop check for what values of `SEQUENCE_LEN` you can train the model successfully.
3. Replace LSTM by a classic RNN and check for what values of `SEQUENCE_LEN` you can train the model successfully.
4. Write a proper curricullum learning loop, where in a loop you consider longer and longer sequences, where expansion of the sequence length happens only after the model is trained successfully on the current length.

Note that for steps 2-4 you may need to change the value of `num_steps`.


In [8]:
class Model2(nn.Module):

    def __init__(self, hidden_dim):
        super(Model2, self).__init__()
        self.hidden_dim = hidden_dim
        self.rnn = nn.RNN(1, self.hidden_dim)
        self.hidden2label = nn.Linear(hidden_dim, 1)

    def forward(self, x):
        out, _ = self.rnn(x)
        sequence_len = x.shape[0]
        logits = self.hidden2label(F.relu(out[-1].view(-1)))
        return logits

In [31]:
SEQUENCE_LEN = 50

# Pairs of (sequence, label)
HARD_EXAMPLES = [([0.]+(SEQUENCE_LEN-1)*[1.], 0),
                 (SEQUENCE_LEN*[1.], 1)]

def eval_on_hard_examples(model, curr_len, silent=False):
    with torch.no_grad():
        logits = []
        for sequence, _ in HARD_EXAMPLES:
            input = torch.tensor(sequence[:curr_len]).view(-1, 1, 1)
            logit = model(input)
            logits.append(logit.detach())

        if not silent:
            print(f'Logits for hard examples={logits}')

    return logits

def train_model(hidden_dim, lr, num_steps=10000):
    model = Model(hidden_dim=hidden_dim)
    loss_function = nn.BCEWithLogitsLoss()
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.99)

    curr_len = 1
    for step in range(num_steps):  
        if step % 100 == 0:
            eval_on_hard_examples(model, curr_len)

        for sequence, label in HARD_EXAMPLES:
            model.zero_grad()

            sequence = sequence[:curr_len]
            perm = torch.randperm(curr_len)
            logit = model(torch.tensor(sequence)[perm].view(-1, 1, 1))  
            
            loss = loss_function(logit.view(-1), torch.tensor([label], dtype=torch.float32))
            loss.backward()

            optimizer.step()

        logits = eval_on_hard_examples(model, curr_len, True)
        if logits[0] * logits[1] < -0.0001 and curr_len < SEQUENCE_LEN:
          curr_len += 1
          print(f"curr_len = {curr_len}")

    print('\nFinal evaluation')
    eval_on_hard_examples(model, SEQUENCE_LEN)

In [35]:
train_model(hidden_dim=20, lr=0.01, num_steps=10_000)

Logits for hard examples=[tensor([0.1503]), tensor([0.1552])]
curr_len = 2
curr_len = 3
curr_len = 4
curr_len = 5
curr_len = 6
curr_len = 7
curr_len = 8
curr_len = 9
curr_len = 10
curr_len = 11
curr_len = 12
curr_len = 13
curr_len = 14
Logits for hard examples=[tensor([-2.9523]), tensor([-2.9555])]
Logits for hard examples=[tensor([-0.3153]), tensor([-0.3153])]
Logits for hard examples=[tensor([0.2032]), tensor([0.2032])]
Logits for hard examples=[tensor([5.4121e-05]), tensor([0.0001])]
curr_len = 15
Logits for hard examples=[tensor([0.7946]), tensor([0.8623])]
curr_len = 16
curr_len = 17
curr_len = 18
curr_len = 19
curr_len = 20
curr_len = 21
curr_len = 22
curr_len = 23
curr_len = 24
curr_len = 25
curr_len = 26
curr_len = 27
curr_len = 28
curr_len = 29
curr_len = 30
curr_len = 31
curr_len = 32
curr_len = 33
Logits for hard examples=[tensor([-3.4395]), tensor([-0.8610])]
curr_len = 34
curr_len = 35
curr_len = 36
curr_len = 37
curr_len = 38
curr_len = 39
curr_len = 40
curr_len = 41
curr